# **GLoVE+BiLSTM**

In [71]:
!pip install keras
!pip install emoji --upgrade
!pip install keras-self-attention
!pip install wordninja

Requirement already up-to-date: emoji in /usr/local/lib/python3.6/dist-packages (0.6.0)


# **Headers**

In [72]:
##################Headers#########
import pandas as pd
import keras
import emoji
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints
from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, Dropout
import numpy as np
import pandas as pd
from keras.callbacks import EarlyStopping, ModelCheckpoint

# **Loading the Data**

In [73]:
from google.colab import files
uploaded = files.upload()

Saving ISEAR.csv to ISEAR (2).csv


In [74]:
import pandas as pd
df=pd.read_csv("ISEAR.csv",names=["emotion","text",""])
df=df[["text","emotion"]]
import numpy as np
from sklearn.model_selection import train_test_split
df_train,df_test=train_test_split(df, test_size=0.25, random_state=42, stratify=df["emotion"])

In [75]:
df_test

,text,emotion
824,Being slandered by friends.,anger
590,I said nasty things which I did not mean to a ...,guilt
2992,I felt it when I came home after the examinati...,guilt
3095,I was disgusted by my paranoid attitude toward...,disgust
3279,As a result of breaking the indicator light as...,fear
...,...,...
2516,When I thought for an instant that a very good...,sadness
1387,When the grandmother of my friend died in a ho...,sadness
2618,I didn't do an important job and it had an adv...,shame
800,The negative feelings towards me displayed by ...,guilt


In [76]:
classes_list = df_train.emotion.unique().tolist()
df_train["class"]=df_train['emotion'].apply(classes_list.index)
df_test["class"]=df_test['emotion'].apply(classes_list.index)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


# ***Preprocessing Functions***

Expanding Contractions


In [77]:
import re
#########Contraction List#####
cList = {"ain't": "am not","aren't": "are not","can't": "cannot","can't've": "cannot have","'cause": "because","could've": "could have",
"couldn't": "could not","couldn't've": "could not have","didn't": "did not","doesn't": "does not","don't": "do not","hadn't": "had not",
"hadn't've": "had not have","hasn't": "has not","haven't": "have not","he'd": "he would","he'd've": "he would have","he'll": "he will","he'll've": "he will have",
"he's": "he is","how'd": "how did","how'd'y": "how do you","how'll": "how will","how's": "how is","I'd": "I would","I'd've": "I would have",
"I'll": "I will","I'll've": "I will have","I'm": "I am","I've": "I have","isn't": "is not","it'd": "it had","it'd've": "it would have",
"it'll": "it will","it'll've": "it will have","it's": "it is","let's": "let us","ma'am": "madam","mayn't": "may not",
"might've": "might have","mightn't": "might not","mightn't've": "might not have","must've": "must have","mustn't": "must not",
"mustn't've": "must not have","needn't": "need not","needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
"oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not","shan't've": "shall not have","she'd": "she would",
"she'd've": "she would have","she'll": "she will","she'll've": "she will have","she's": "she is","should've": "should have",
"shouldn't": "should not","shouldn't've": "should not have","so've": "so have","so's": "so is","that'd": "that would",
"that'd've": "that would have","that's": "that is","there'd": "there had","there'd've": "there would have","there's": "there is",
"they'd": "they would","they'd've": "they would have","they'll": "they will","they'll've": "they will have","they're": "they are",
"they've": "they have","to've": "to have","wasn't": "was not","we'd": "we had","we'd've": "we would have","we'll": "we will",
"we'll've": "we will have","we're": "we are","we've": "we have","weren't": "were not","what'll": "what will","what'll've": "what will have",
"what're": "what are","what's": "what is","what've": "what have","when's": "when is","when've": "when have","where'd": "where did",
"where's": "where is","where've": "where have","who'll": "who will","who'll've": "who will have","who's": "who is","who've": "who have",
"why's": "why is","why've": "why have","will've": "will have","won't": "will not","won't've": "will not have","would've": "would have",
"wouldn't": "would not","wouldn't've": "would not have","y'all": "you all","y'alls": "you alls","y'all'd": "you all would",
"y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you had","you'd've": "you would have",
"you'll": "you you will","you'll've": "you you will have","you're": "you are","you've": "you have"}
#########Function for contraction#####
c_re = re.compile('(%s)' % '|'.join(cList.keys())) #regular expression object, with format of contractions like i'm, i've
def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)] #find the match in the list
    return c_re.sub(replace, text)  #replacing the short hands with expanded version

Lemmatization according to sentence structure 

In [78]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [79]:
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
from collections import defaultdict
from nltk.corpus import stopwords
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
lemma_function = WordNetLemmatizer()
def lemmatize(row):
  tokens = word_tokenize(row)
  new =  ""
  for token, tag in pos_tag(tokens):
    if token not in stopwords.words('english'):
      lemma = lemma_function.lemmatize(token, tag_map[tag[0]])
      new=new+lemma+" "
  return new

In [80]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# **Preprocessing**

In [81]:
from nltk.stem import WordNetLemmatizer
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
df_train["text"]=df_train["text"].str.replace("@","username") #removing @
df_train["text"]=df_train["text"].str.replace("'s","")
df_train["text"]=df_train["text"].str.replace("?"," question ")
df_train["text"]=df_train["text"].str.replace("!"," exclamation ")
df_train["text"]=df_train["text"].str.replace(r'http\S+', '')# removing links
df_train["text"]=df_train["text"].apply(expandContractions) #expanding contractions
df_train["text"]=df_train["text"].apply(emoji.demojize)#changing emojis to text
df_train["text"]=df_train["text"].str.replace("#","").str.replace("_"," ").str.replace(":"," ").str.lower()
df_train["text"]=df_train["text"].apply(lemmatize) #lemmatization
df_train["text"]=df_train["text"].str.replace(r'[^\w\s]'," ") #removing punctuation
df_test["text"]=df_test["text"].str.replace("@","username")
df_test["text"]=df_test["text"].str.replace("'s","")
df_test["text"]=df_test["text"].str.replace("?"," question ")
df_test["text"]=df_test["text"].str.replace("!"," exclamation ")
df_test["text"]=df_test["text"].str.replace(r'http\S+', '')
df_test["text"]=df_test["text"].apply(expandContractions)
df_test["text"]=df_test["text"].apply(emoji.demojize)
df_test["text"]=df_test["text"].str.replace("#","").str.replace("_"," ").str.replace(":","").str.lower()
df_test["text"]=df_test["text"].apply(lemmatize)
df_test["text"]=df_test["text"].str.replace(r'[^\w\s]',"")
tokenizer = Tokenizer(num_words=5000) # tokenizing
tokenizer.fit_on_texts(df_train["text"].values.tolist())
#Creating the training and testing sets consisting of tokens
X_train = tokenizer.texts_to_sequences(df_train["text"].values.tolist())
X_test = tokenizer.texts_to_sequences(df_test["text"].values.tolist())
vocab_size = len(tokenizer.word_index) + 1 
word_index=tokenizer.word_index

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

In [82]:
len(word_index)

6260

Preparing the training and testing sets for Classification

In [83]:
maxlen=max([len(a) for a in X_train])

In [84]:
from keras.preprocessing.sequence import pad_sequences
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [85]:
y_train=df_train["class"].values.tolist()

# **Function for Embedding**

In [48]:
import numpy as np
def make_glovevec(glovepath, max_features, embed_size, word_index, veclen=300):
    embeddings_index = {}
    f = open(glovepath)
    for line in f:
        values = line.split()
        dimension=len(values)-1
        word = ' '.join(values[:-300])
        coefs = np.asarray(values[-300:], dtype='float32')
        embeddings_index[word] = coefs.reshape(-1)
    f.close()

    nb_words = min(max_features, len(word_index)+1)
    embedding_matrix = np.zeros((nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [49]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
max_features=len(word_index)
embed_size=300
embedding_vector = make_glovevec("/content/drive/My Drive/glove.840B.300d.txt", max_features+1, embed_size, word_index)

# The Classifiers 

In [52]:
 ############### Keras
from keras.models import Sequential
from keras_self_attention import SeqSelfAttention as Attention
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import confusion_matrix

In [53]:
np.unique(y_train)

array([0, 1, 2, 3, 4, 5, 6])

In [86]:
from sklearn.utils import class_weight
#################BiLSTM Attention+ Relu without any dropouts ######################
def make_model():
  d2=0.0
  d1=0.0
  #class_weight=class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
  model_glove = Sequential()
  model_glove.add(Embedding(len(word_index)+1, 300, input_length=78, weights=[embedding_vector], trainable=False))
  model_glove.add(Bidirectional(LSTM(300, return_sequences=True, dropout=d1,recurrent_dropout=d2)))
  #model_glove.add(Attention(attention_activation='sigmoid'))
  #model_glove.add(Flatten())
  model_glove.add(Dense(10, activation='relu'))
  model_glove.add(Flatten())
  #model_glove.add(Dropout(0.0)) 
  model_glove.add(Dense(7, activation='softmax'))
  #model_glove.add(Flatten())
  return model_glove

In [87]:
from sklearn.utils import class_weight
#################BiLSTM Attention+ Relu without any dropouts ######################
def make_model_lstm():
  d2=0.0
  d1=0.0
  #class_weight=class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
  model_glove = Sequential()
  model_glove.add(Embedding(len(word_index)+1, 300, input_length=78, weights=[embedding_vector], trainable=False))
  model_glove.add(Bidirectional(LSTM(300, return_sequences=True, dropout=d1,recurrent_dropout=d2)))
  #model_glove.add(Attention(attention_activation='sigmoid'))
  #model_glove.add(Flatten())
  model_glove.add(Dense(10, activation='relu'))
  model_glove.add(Flatten())
  #model_glove.add(Dropout(0.0)) 
  model_glove.add(Dense(7, activation='softmax'))
  #model_glove.add(Flatten())
  return model_glove

In [88]:
import os
import tensorflow as tf
#resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
#tf.config.experimental_connect_to_host(resolver.master())
#tf.tpu.experimental.initialize_tpu_system(resolver)
#strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [89]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

strategy = tf.distribute.experimental.TPUStrategy(tpu)
print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  ['10.121.28.138:8470']


INFO:tensorflow:Initializing the TPU system: grpc://10.121.28.138:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.121.28.138:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [90]:
with strategy.scope():
  model_glove=make_model()
  opt = keras.optimizers.Adam(learning_rate=0.00001)
  model_glove.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model_glove.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 78, 300)           1878300   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 78, 600)           1442400   
_________________________________________________________________
dense_6 (Dense)              (None, 78, 10)            6010      
_________________________________________________________________
flatten_3 (Flatten)          (None, 780)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 7)                 5467      
Total params: 3,332,177
Trainable params: 1,453,877
Non-trainable params: 1,878,300
_________________________________________________________________


In [92]:
from keras.utils import to_categorical
y_binary = to_categorical(y_train)
model_glove.fit(X_train, y_binary, epochs =15)#,class_weight = class_weight)#,sample_weight=sample_weight)
model_glove.summary()


Epoch 1/15
  1/177 [..............................] - ETA: 4:15 - loss: 1.9477 - accuracy: 0.1250WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0025s vs `on_train_batch_end` time: 0.0244s). Check your callbacks.


177/177 [==============================] - 7s 40ms/step - loss: 1.9408 - accuracy: 0.1689
Epoch 2/15
177/177 [==============================] - 5s 27ms/step - loss: 1.9258 - accuracy: 0.2395
Epoch 3/15
177/177 [==============================] - 5s 27ms/step - loss: 1.9057 - accuracy: 0.3120
Epoch 4/15
177/177 [==============================] - 5s 27ms/step - loss: 1.8692 - accuracy: 0.3472
Epoch 5/15
177/177 [==============================] - 5s 27ms/step - loss: 1.7927 - accuracy: 0.3502
Epoch 6/15
177/177 [==============================] - 5s 27ms/step - loss: 1.7119 - accuracy: 0.3404
Epoch 7/15
177/177 [==============================] - 5s 27ms/step - loss: 1.6658 - accuracy: 0.3507
Epoch 8/15
177/177 [==============================] - 5s 27ms/step - loss: 1.6341 - accuracy: 0.3651
Epoch 9/15
177/177 [==============================] - 5s 27ms/step - loss: 1.6050 - accuracy: 0.3789
Epoch 10/15
177/177 [==============================] - 5s 27ms/step - loss: 1.5772 - accuracy: 0.3915


In [94]:
y_test=df_test["class"].values.tolist()
#y_pred= model_glove.predict_classes(X_test)
y_pred= np.argmax(model_glove.predict(X_test),axis=-1)
print("Accuracy score =", accuracy_score(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred, target_names=classes_list))
confusion_matrix(y_test, y_pred)   

Accuracy score = 0.41351782863225117
              precision    recall  f1-score   support

        fear       0.60      0.55      0.57       269
       anger       0.34      0.30      0.32       270
       guilt       0.31      0.17      0.22       262
     sadness       0.37      0.43      0.40       271
     disgust       0.43      0.44      0.43       266
       shame       0.31      0.25      0.28       268
         joy       0.45      0.75      0.56       273

    accuracy                           0.41      1879
   macro avg       0.40      0.41      0.40      1879
weighted avg       0.40      0.41      0.40      1879



array([[147,  18,   9,  40,  26,   5,  24],
       [ 23,  80,  25,  22,  48,  44,  28],
       [ 22,  48,  45,  41,  21,  43,  42],
       [ 11,  13,  15, 117,  15,   8,  92],
       [ 29,  24,  11,  28, 116,  40,  18],
       [  8,  41,  36,  32,  41,  68,  42],
       [  6,   8,   6,  33,   3,  13, 204]])

In [95]:
with strategy.scope():
  model_glove=make_model_lstm()
  opt = keras.optimizers.Adam(learning_rate=0.00001)
  model_glove.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model_glove.summary()
from keras.utils import to_categorical
y_binary = to_categorical(y_train)
model_glove.fit(X_train, y_binary, epochs =15)#,class_weight = class_weight)#,sample_weight=sample_weight)
model_glove.summary()
y_test=df_test["class"].values.tolist()
#y_pred= model_glove.predict_classes(X_test)
y_pred= np.argmax(model_glove.predict(X_test),axis=-1)
print("Accuracy score =", accuracy_score(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred, target_names=classes_list))
confusion_matrix(y_test, y_pred)   

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 78, 300)           1878300   
_________________________________________________________________
bidirectional_4 (Bidirection (None, 78, 600)           1442400   
_________________________________________________________________
dense_8 (Dense)              (None, 78, 10)            6010      
_________________________________________________________________
flatten_4 (Flatten)          (None, 780)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 7)                 5467      
Total params: 3,332,177
Trainable params: 1,453,877
Non-trainable params: 1,878,300
_________________________________________________________________
Epoch 1/15
  1/177 [..............................] - ETA: 4:11 - loss: 1.9472 - accuracy: 0.1250WARNI

177/177 [==============================] - 7s 40ms/step - loss: 1.9445 - accuracy: 0.1600
Epoch 2/15
177/177 [==============================] - 5s 27ms/step - loss: 1.9362 - accuracy: 0.2113
Epoch 3/15
177/177 [==============================] - 5s 28ms/step - loss: 1.9222 - accuracy: 0.2143
Epoch 4/15
177/177 [==============================] - 5s 27ms/step - loss: 1.8943 - accuracy: 0.2225
Epoch 5/15
177/177 [==============================] - 5s 27ms/step - loss: 1.8151 - accuracy: 0.2611
Epoch 6/15
177/177 [==============================] - 5s 27ms/step - loss: 1.7230 - accuracy: 0.3000
Epoch 7/15
177/177 [==============================] - 5s 27ms/step - loss: 1.6842 - accuracy: 0.3168
Epoch 8/15
177/177 [==============================] - 5s 27ms/step - loss: 1.6630 - accuracy: 0.3452
Epoch 9/15
177/177 [==============================] - 5s 28ms/step - loss: 1.6429 - accuracy: 0.3614
Epoch 10/15
177/177 [==============================] - 5s 27ms/step - loss: 1.6213 - accuracy: 0.3830


array([[101,  17,  16,  39,  62,  10,  24],
       [ 19,  65,  28,  16,  89,  30,  23],
       [ 21,  45,  59,  37,  53,  29,  18],
       [ 18,  18,  17, 137,  16,  11,  54],
       [ 14,  19,  15,  17, 166,  22,  13],
       [ 11,  43,  28,  20,  81,  58,  27],
       [  3,   9,  20,  39,  13,  11, 178]])